# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [43]:


# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas

import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
import panel as pn

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

%matplotlib inline

In [25]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [26]:
# Import the CSVs to Pandas DataFrames
file_path = Path("C:/Users/14259/Desktop/Kevin/UW/FinTech-Bootcamp/homework/Unit-6-PyViz/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("C:/Users/14259/Desktop/Kevin/UW/FinTech-Bootcamp/homework/Unit-6-PyViz/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [73]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units = sfo_data.groupby('year').mean()
    housing_units.drop(['sale_price_sqr_foot', 'gross_rent'], axis =1, inplace = True)
    
    
    minimum = housing_units.min()['housing_units'] - 2000
    maximum = housing_units.max()['housing_units'] + 2000
    
    housing_units_plot = housing_units.plot(legend = False,kind = 'bar',ylim = [minimum, maximum],title = "Housing Units in San Francisco from 2010 to 2016")
    
    return housing_units_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    avg_rent_sqft = sfo_data.groupby('year').mean()
    avg_rent_sqft.drop(['housing_units'], axis =1, inplace = True)
    avg_gross_rent = avg_rent_sqft['gross_rent'].plot(title = 'Average Gross Rent in San Francisco')
    plt.xlabel('Year')
    plt.ylabel('Gross Rent')
    plt.close()
    
    return pn.pane.Matplotlib(avg_rent_sqft)


def average_sales_price():
    """Average Sales Price Per Year."""
    
    avg_rent_sqft = sfo_data.groupby('year').mean()
    avg_rent_sqft.drop(['housing_units'], axis =1, inplace = True)
    
    avg_sales_price = avg_rent_sqft['sale_price_sqr_foot'].plot(title = 'Average Sale Price per Square Foot in San Francisco')
    plt.xlabel('Year')
    plt.ylabel('Avg. Sale Price')
    
    return avg_sales_price


def average_price_by_neighborhood(neighborhood):
    """Average Prices by Neighborhood."""
    
    sfo_data1 = sfo_data.groupby([sfo_data.index, 'neighborhood']).mean()
    sfo_data2 = pd.DataFrame(sfo_data1).reset_index()
    avg_price = sfo_data2.loc[sfo_data2['neighborhood'] == neighborhood].hvplot.line(
        x='year',
        y='sale_price_sqr_foot',
        title='San Francisco Sales Per SQFT per Year'
    )
    
    return avg_price

sfo_data1 = sfo_data.groupby([sfo_data.index, 'neighborhood']).mean()
sfo_data2 = pd.DataFrame(sfo_data1).reset_index()
neighborhood_df = sfo_data2['neighborhood']
interact(average_price_by_neighborhood, neighborhood = neighborhood_df)

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    top_10 = sfo_data.groupby(['neighborhood']).mean()
    top_10_data = pd.DataFrame(top_10).sort_values('sale_price_sqr_foot', ascending = False).head(10).reset_index()
    top_10_bar = top_10_data.hvplot.bar(
    x='neighborhood',
    y='sale_price_sqr_foot',
    title='Top 10 Most Expensive Neighborhoods',
    rot = 45
    
    )
    
    return top_10_bar

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    top_10 = sfo_data.groupby(['neighborhood']).mean()
    top_10_data = pd.DataFrame(top_10).sort_values('sale_price_sqr_foot', ascending = False).head(10).reset_index()
    top_10_parallel = px.parallel_coordinates(top_10_data, color='sale_price_sqr_foot')
    
    return top_10_parallel
    
def parallel_categories():
    """Parallel Categories Plot."""
    
    top_10 = sfo_data.groupby(['neighborhood']).mean()
    top_10_data = pd.DataFrame(top_10).sort_values('sale_price_sqr_foot', ascending = False).head(10).reset_index()
    top_10_par_categories = px.parallel_categories(top_10_data, dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'], color='sale_price_sqr_foot')
    
    
    return top_10_par_categories

def neighborhood_map():
    """Neighborhood Map"""
    
    sfo_average = sfo_data.groupby(['neighborhood']).mean()
    sfo_average = pd.DataFrame(sfo_average).reset_index()
    
    avg_value = pd.concat([sfo_average, df_neighborhood_locations], axis ='columns', join = 'inner')
    sfo_map = px.scatter_mapbox(avg_value,lat='Lat',lon='Lon',size='sale_price_sqr_foot',color='gross_rent',title='Aerage Sale Price Per Square Foot and Gross Rent in San Francisco')
    
    return sfo_map



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [76]:
welcome_tab = pn.Row(pn.Column(title), neighborhood_map())

neighborhood_analysis_tab = pn.Column(interact(average_price_by_neighborhood, neighborhood = neighborhood_df),
    top_most_expensive_neighborhoods(), neighborhood_map())

averages_tab = pn.Column(parallel_categories(), parallel_coordinates())

# Create tabs

panel = pn.Tabs(
("Neighborhood Analysis", neighborhood_analysis_tab), 
("Parallel Charts", parallel_plots_tab))



## Serve the Panel Dashboard

In [77]:
panel.servable()

Tabs
    [0] Column
        [0] Column
            [0] Column
                [0] Select(name='neighborhood', options=['Alamo Square', ...], value='Alamo Square')
            [1] Row
                [0] HoloViews(Curve, name='interactive11114')
        [1] HoloViews(Bars)
        [2] Plotly(Figure)
    [1] Column
        [0] Str(AxesSubplot)